In [2]:
import ctypes
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# ======================= PMC Wrapper =======================
class PMCConfig(ctypes.Structure):
    _fields_ = [
        ("n_inputs", ctypes.c_uint),
        ("n_hidden", ctypes.c_uint),
        ("n_outputs", ctypes.c_uint),
        ("learning_rate", ctypes.c_double)
    ]

class PMC:
    def __init__(self, n_inputs, n_hidden, n_outputs=1, learning_rate=0.01):
        dll_path = "./target/release/neural_networks.dll"
        print(f"Chargement de: {dll_path}")
        self.lib = ctypes.CDLL(dll_path)

        # Fonctions du DLL
        self.lib.pmc_new.argtypes = [ctypes.POINTER(PMCConfig)]
        self.lib.pmc_new.restype = ctypes.c_void_p
        self.lib.pmc_delete.argtypes = [ctypes.c_void_p]

        self.lib.pmc_fit.argtypes = [
            ctypes.c_void_p,
            ctypes.POINTER(ctypes.c_double),
            ctypes.POINTER(ctypes.c_double),
            ctypes.c_size_t,
            ctypes.c_size_t,
            ctypes.c_size_t
        ]
        self.lib.pmc_fit.restype = ctypes.c_double

        self.lib.pmc_predict_batch.argtypes = [
            ctypes.c_void_p,
            ctypes.POINTER(ctypes.c_double),
            ctypes.POINTER(ctypes.c_double),
            ctypes.c_size_t,
            ctypes.c_size_t
        ]

        self.lib.pmc_accuracy.argtypes = [
            ctypes.c_void_p,
            ctypes.POINTER(ctypes.c_double),
            ctypes.POINTER(ctypes.c_double),
            ctypes.c_size_t,
            ctypes.c_size_t
        ]
        self.lib.pmc_accuracy.restype = ctypes.c_double

        # Config
        self.config = PMCConfig(
            n_inputs=n_inputs,
            n_hidden=n_hidden,
            n_outputs=n_outputs,
            learning_rate=learning_rate
        )

        self.ptr = self.lib.pmc_new(ctypes.byref(self.config))
        if not self.ptr:
            raise RuntimeError("Échec de création du modèle PMC")

        self.n_inputs = n_inputs
        self.n_outputs = n_outputs

    def __del__(self):
        if hasattr(self, 'ptr') and self.ptr:
            self.lib.pmc_delete(self.ptr)

    def fit(self, X, y, max_iterations=1000):
        n_samples, n_features = X.shape
        if n_features != self.n_inputs:
            raise ValueError(f"Attendu {self.n_inputs} features, reçu {n_features}")

        X_flat = X.astype(np.float64).flatten()
        y_flat = y.astype(np.float64).flatten()

        X_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
        y_ptr = y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

        error = self.lib.pmc_fit(self.ptr, X_ptr, y_ptr, n_samples, n_features, max_iterations)
        return error

    def predict(self, X):
        n_samples, n_features = X.shape
        if n_features != self.n_inputs:
            raise ValueError(f"Attendu {self.n_inputs} features, reçu {n_features}")

        X_flat = X.astype(np.float64).flatten()
        results = np.zeros(n_samples * self.n_outputs, dtype=np.float64)

        X_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
        results_ptr = results.ctypes.data_as(ctypes.POINTER(ctypes.c_double))

        self.lib.pmc_predict_batch(self.ptr, X_ptr, results_ptr, n_samples, n_features)

        if self.n_outputs > 1:
            results = results.reshape(n_samples, self.n_outputs)
        return results

    def predict_class(self, X, threshold=0.0):
        predictions = self.predict(X)
        if self.n_outputs == 1:
            return np.where(predictions >= threshold, 1, -1)
        else:
            return np.argmax(predictions, axis=1)

    def accuracy(self, X, y):
        n_samples, n_features = X.shape
        if n_features != self.n_inputs:
            raise ValueError(f"Attendu {self.n_inputs} features, reçu {n_features}")

        X_flat = X.astype(np.float64).flatten()
        y_flat = y.astype(np.float64).flatten()

        X_ptr = X_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
        y_ptr = y_flat.ctypes.data_as(ctypes.POINTER(ctypes.c_double))
        return self.lib.pmc_accuracy(self.ptr, X_ptr, y_ptr, n_samples, n_features)

# ======================= Visualisation =======================
def visualiser_frontiere_decision(X, Y, model, titre, threshold=0.0):
    if X.shape[1] != 2:
        print("Visualisation disponible seulement pour 2 features.")
        return

    h = 0.02
    x_min, x_max = X[:, 0].min() - 0.5, X[:, 0].max() + 0.5
    y_min, y_max = X[:, 1].min() - 0.5, X[:, 1].max() + 0.5
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))

    Z = model.predict_class(np.c_[xx.ravel(), yy.ravel()], threshold)
    Z = Z.reshape(xx.shape)

    plt.figure(figsize=(8, 6))

    if model.n_outputs == 1:
        plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.RdBu)
        plt.scatter(X[Y == 1, 0], X[Y == 1, 1], color='blue', label='Classe 1', edgecolors='k')
        plt.scatter(X[Y == -1, 0], X[Y == -1, 1], color='red', label='Classe -1', edgecolors='k')
    else:
        plt.contourf(xx, yy, Z, alpha=0.3, cmap=plt.cm.coolwarm)
        unique_classes = np.unique(Y)
        colors = ['blue', 'red', 'green', 'orange', 'purple']
        for i, cls in enumerate(unique_classes):
            if i < len(colors):
                plt.scatter(X[Y == cls, 0], X[Y == cls, 1],
                            color=colors[i], label=f'Classe {int(cls)}', edgecolors='k')

    plt.title(titre)
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.legend()
    plt.show()
